In [ ]:
!pip -q install cadquery trimesh plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.6/182.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 740.3/740.3 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 106.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.2/440.2 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.3/82.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 539.8/539.8 kB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 99.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.1/245.1 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 2.8 MB/s eta 0:00:00


In [ ]:
import cadquery as cq
from cadquery import exporters
import os, trimesh
import plotly.graph_objects as go

def export_stl(shape, filename, tol=0.08):
    exporters.export(shape, filename, tolerance=tol, angularTolerance=0.3)
    print("Wrote", filename, "bytes", os.path.getsize(filename) if os.path.exists(filename) else None)

def show_stl(filename, with_edges=False):
    m = trimesh.load(filename)
    V = m.vertices

    data = [go.Mesh3d(
        x=V[:,0], y=V[:,1], z=V[:,2],
        i=m.faces[:,0], j=m.faces[:,1], k=m.faces[:,2],
        opacity=1.0
    )]

    if with_edges:
        E = m.edges_unique
        xs, ys, zs = [], [], []
        for a,b in E:
            xs += [V[a,0], V[b,0], None]
            ys += [V[a,1], V[b,1], None]
            zs += [V[a,2], V[b,2], None]
        data.append(go.Scatter3d(x=xs, y=ys, z=zs, mode="lines", line=dict(width=1)))

    fig = go.Figure(data=data)
    fig.update_layout(scene_aspectmode="data", margin=dict(l=0,r=0,t=0,b=0))
    fig.show()

In [ ]:
# Params
major_d = 20.0
body_len = 60.0
body_r = major_d/2

head_d = 28.0
head_h = 3.0

# Create
body = cq.Workplane("XY").circle(body_r).extrude(body_len)
head = cq.Workplane("XY").circle(head_d/2).extrude(head_h).val().translate((0,0,body_len))

blank = body.val().fuse(head)

export_stl(blank, "step1_blank.stl")
show_stl("step1_blank.stl", with_edges=False)

Wrote step1_blank.stl bytes 16684


In [ ]:
shape = blank

n = 24        # 溝の本数（多いほどギザギザ）
cut_w = 1.0   # 溝の幅
cut_d = 1.2   # 溝の深さ（ここ増やすとギザギザ強くなる）
z0 = body_len

for i in range(n):
    ang = 360.0 * i / n
    cutter = (
        cq.Workplane("XY")
        .center(head_d/2 - cut_d/2, 0)  # ヘッド外周近く
        .rect(cut_d, cut_w)
        .extrude(head_h)
        .val()
        .rotate((0,0,0), (0,0,1), ang)
        .translate((0,0,z0))
    )
    shape = shape.cut(cutter)

export_stl(shape, "step2_head_knurl.stl")
show_stl("step2_head_knurl.stl", with_edges=False)

Wrote step2_head_knurl.stl bytes 37084


In [ ]:
shape3 = shape

slot_w = 6.0
slot_d = 2.8
slot_l = head_d * 1.2
z_top = body_len + head_h

slot1 = cq.Workplane("XY").rect(slot_l, slot_w).extrude(slot_d).val().translate((0,0,z_top-slot_d))
slot2 = cq.Workplane("XY").rect(slot_w, slot_l).extrude(slot_d).val().translate((0,0,z_top-slot_d))

shape3 = shape3.cut(slot1).cut(slot2)

export_stl(shape3, "step3_phillips.stl")
show_stl("step3_phillips.stl", with_edges=False)

Wrote step3_phillips.stl bytes 44284


In [ ]:
#shape4 = shape3

pitch = 5.0           # ★高さ（間隔）を出す
groove_r = 7
helix_r  = 14        # ★切り込み場所（body_r=10 なら invasion=0.5mm）

# 結果を表示
r_inner = helix_r - groove_r
r_outer = helix_r + groove_r
invasion = max(0.0, body_r - r_inner)
outer    = max(0.0, r_outer - body_r)
print("invasion(mm)=", invasion, " outer(mm)=", outer)

helix = cq.Wire.makeHelix(pitch=pitch, height=body_len, radius=helix_r)
cutter = cq.Workplane("XY").center(helix_r, 0).circle(groove_r).sweep(helix, isFrenet=True).val()

shape4 = shape3.cut(cutter)

export_stl(shape4, "step4_roundgroove_independent.stl")
show_stl("step4_roundgroove_independent.stl", with_edges=False)

invasion(mm)= 3.0  outer(mm)= 11.0
Wrote step4_roundgroove_independent.stl bytes 586834
